# Python Optimization Overview

## Outline

1. Server Configuration
2. Optimization Views
    - Retrieve an Optimization View
    - List Optimization Views
3. Optimization Domains
4. Candidates
    - Retrieve Candidates by Index
    - List Candidate Parameters
    - Retrieve Candidate Metrics
    - Perform metric requests on the Candidate Portfolios
5. Initial Portfolio Metrics
6. Retrieve Sensitivity Analysis
    - Filter by Candidates' Indexes
7. Download Optimization Results CSV

**NOTE: The code snippets in this notebook file can be found in http://docs.analyzere.net/#optimization-views by navigating through the sub-sections on the left pane and choosing the Python language on the code snippet area.**

In [5]:
import analyzere
import csv

from getpass import getpass
from analyzere import OptimizationView, PortfolioView
from analyzere.requestor import request_raw, request
from analyzere.base_resources import convert_to_analyzere_object
from analyzere.resources import Candidate

### 1. Sever Configuration

In [ ]:
# Update the correct URL and username below and run the cell to provide the password

analyzere.base_url = 'https://yourcompany-api.analyzere.net/'
analyzere.username = ''
analyzere.password = getpass('Password: ')

### 2. Optimization Views

Optimization Views provide the ability to execute optimization of portfolios in the AnalyzeRe system.

#### Retrieving an Optimization View

In [ ]:
ov = analyzere.OptimizationView.retrieve('cf582b25-553f-4f2e-9090-0059ca73ce52')
print('Optimization View: {}'.format(ov.description))

#### Listing Optimization Views

In [ ]:
optimization_views = analyzere.OptimizationView.list(search='demo',
                                           ordering='-created')

print('Retrieved {} Optimization Views'.format(len(optimization_views)))

for optimization_view in optimization_views:
    print('{} : {} '.format(optimization_view.id, optimization_view.description))

### 3. Optimization Domains

Optimization Domains that specify which Layers to include, along with per-layer constraints.

In [ ]:
print('Total Optimization Domains in this OptimizationView: {}'.format(len(ov.domains)))

for domain in ov.domains:
    print('Sample domain: {}'.format(domain))

### 4. Candidates

When an optimization is finished running, its candidates are available. They provide the optimized participation values for the Layers specified as Optimization Domains.

In [ ]:
candidates = ov.candidates()
print('Total number of candidates: {}'.format(len(candidates)))

for c in candidates:
    print('Candidate Portfolio: {}'.format(c.portfolio_view().id))
    break # Remove this line to print all the Candidate Portfolios

#### Retrieve Candidates by Index

Candidates can be retrieved by index. 
Note that you’ll receive a 404 Not Found error if requesting any candidates 
outside the range of candidates available.

In [ ]:
# Retrieving first candidate

candidate = ov.candidates(0)
print(candidate)

####  List Candidate Parameters

Retrieves the parameterizations of candidates. It returns all candidates’ layer participation values in an array. The order of the values within the participations array corresponds to the layers’ indexes in the OptimizationDomain of the OptimizationView.

In [ ]:
candidate_parameters = ov.candidate_parameters()

print(candidate_parameters)

#### Retrieve Candidate Metrics

Returns the objective, constraint, and feasibility values for all candidates as lists that are ordered by the index of their corresponding candidate.

In [ ]:
candidate_metrics = ov.candidate_metrics()

print(candidate_metrics)

#### Retrieve Candidate Portfolio

Presently, there is not a direct method to retrieve a candidate portfolio (given the portfolio view and candidate index) without retrieving the entire Optimization View. However, the following workflow shows how it can be done with a simple `candidate_count` method and some additional code. Please note: it's best to execute this code separately from the rest of the notebook.

In [ ]:
def candidate_count(self):
  path = '{}/candidates'.format(self._get_path(self.id))
  resp = request('get', path, params={"limit": 0})
  return convert_to_analyzere_object(resp, Candidate, optimization_view_id=self.id).meta.total_count

OptimizationView.candidate_count = candidate_count

In [ ]:
ov = OptimizationView.list(ids=["portfolio-view-id here"], fields="id")[0]

Once we selected the `OptimizationView`, we can loop through the total candidates and find the candidate portfolio which is relevant to us.

In [ ]:
for i in range(ov.candidate_count()):
  pv = ov.candidates(i).portfolio_view().id
  print(pv)

#### Perform metric requests on the Candidate Portfolios
Each optimization Candidate provides a convenience access to its PortfolioView. Metrics can be requested to perform detailed analysis on any given candidate via its PortfolioView. For all the available metric requests supported by the Python client library, visit http://docs.analyzere.net/#retrieve223.

In [ ]:
candidates = ov.candidates()

for c in candidates:
    print('Candidate Portfolio: {}'.format(c.portfolio_view().id))
    print('1 in 100 TVAR: {}'.format(c.portfolio_view().tvar(0.1)))
    print('AAL: {}'.format(c.portfolio_view().el()))
    print('Tail Metrics: {}'.format(c.portfolio_view().tail_metrics(0.1)))
    break # Remove this statement to print the results of all the candidates

### 5. Initial Portfolio Metrics

The initial portfolio metrics can be retrieved for an Optimization View to inspect the analysis results for the layers provided at Optimization View creation time. These metrics may be compared to the resulting candidates to determine the suitability of individual candidates.

In [ ]:
initial_portfolio_metrics = ov.initial_metrics()

print(initial_portfolio_metrics)

### 6. Retrieve Sensitivity Analysis


We can analyze the distribution properties of layers participations across all available candidate portfolios. The endpoint returns a list of layer sensitivities. Each layer sensitivity contains several statistical metrics: minimum, maximum and average layer participation; normalized standard deviation and interquartile ranges; and histogram data.

In [ ]:
sensitivity_analysis = ov.sensitivity_analysis()

print(sensitivity_analysis)

#### Filter by Candidates' Indexes

If a list of candidates is not provided, then all candidates are included in the analysis. If a list of candidates is provided, an analysis is performed only on the selected candidates. If a list contains only non-existing candidates, an endpoint will return an empty list.

In [ ]:
candidates_list = [0,1]

sa = ov.sensitivity_analysis(candidates=[candidates_list])
print(sa)

### 7. Download Optimization Results in a CSV

The **optimization_views/{optimization_view_uuid}/csv** endpoint can be used to download the results of an optimization run in a CSV file.

In [ ]:
# Download the CSV

path = "optimization_views/{}/csv".format(ov.id)
csv_content = request_raw("GET", path).text
csv_file_name = '{}_{}.csv'.format(ov.description, ov.id)

with open(csv_file_name, 'w', newline='\n') as output_file:
    output_file.write(csv_content)